In [33]:
import pandas as pd
pd.__version__

import matplotlib.pyplot as plt
%matplotlib inline

# ri stands for Rhode Island
ri = pd.read_csv('police.csv')
ri.head()

,stop_date,stop_time,county_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop
0,2005-01-02,01:55,NaN,M,1985.0,20.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
1,2005-01-18,08:15,NaN,M,1965.0,40.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
2,2005-01-23,23:15,NaN,M,1972.0,33.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
3,2005-02-20,17:15,NaN,M,1986.0,19.0,White,Call for Service,Other,False,NaN,Arrest Driver,True,16-30 Min,False
4,2005-03-14,10:00,NaN,F,1984.0,21.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False


# 1. Remove the column that only contains missing values

In [34]:
ri.isnull().mean()

stop_date             0.000000
stop_time             0.000000
county_name           1.000000
driver_gender         0.058153
driver_age_raw        0.058066
driver_age            0.061270
driver_race           0.058131
violation_raw         0.058131
violation             0.058131
search_conducted      0.000000
search_type           0.965163
stop_outcome          0.058131
is_arrested           0.058131
stop_duration         0.058131
drugs_related_stop    0.000000
dtype: float64

In [35]:
ri.drop('county_name', axis=1)

,stop_date,stop_time,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop
0,2005-01-02,01:55,M,1985.0,20.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
1,2005-01-18,08:15,M,1965.0,40.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
2,2005-01-23,23:15,M,1972.0,33.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
3,2005-02-20,17:15,M,1986.0,19.0,White,Call for Service,Other,False,NaN,Arrest Driver,True,16-30 Min,False
4,2005-03-14,10:00,F,1984.0,21.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91736,2015-12-31,20:27,M,1986.0,29.0,White,Speeding,Speeding,False,NaN,Warning,False,0-15 Min,False
91737,2015-12-31,20:35,F,1982.0,33.0,White,Equipment/Inspection Violation,Equipment,False,NaN,Warning,False,0-15 Min,False
91738,2015-12-31,20:45,M,1992.0,23.0,White,Other Traffic Violation,Moving violation,False,NaN,Warning,False,0-15 Min,False
91739,2015-12-31,21:42,M,1993.0,22.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False


## 1.1 Remove all columns with all values as Nulls

In [36]:
ri.dropna(axis=1, how='all', inplace=True)

## 2.1 Do men or women speed more often?

In [37]:
ri.columns

Index(['stop_date', 'stop_time', 'driver_gender', 'driver_age_raw',
       'driver_age', 'driver_race', 'violation_raw', 'violation',
       'search_conducted', 'search_type', 'stop_outcome', 'is_arrested',
       'stop_duration', 'drugs_related_stop'],
      dtype='object')

In [38]:
print(ri['violation'].unique())
print(ri['driver_gender'].unique())

['Speeding' 'Other' 'Equipment' 'Moving violation' nan
 'Registration/plates' 'Seat belt']
['M' 'F' nan]


In [39]:
ri.groupby('driver_gender')['violation'].value_counts(normalize=True).loc[:, 'Speeding']

driver_gender
F    0.65850
M    0.52435
Name: violation, dtype: float64

## 3. Does gender affect who gets searched during a stop?

### Driver_gender, search_conducted
#### Lesson:

- Causation is difficult to conclude, so focus on realtionhips
- Include all relevant factors when studying a realationship


In [40]:
ri.groupby('driver_gender')['search_conducted'].mean()

driver_gender
F    0.020033
M    0.043326
Name: search_conducted, dtype: float64

In [41]:
ri.groupby(['violation', 'driver_gender'])['search_conducted'].mean()


violation            driver_gender
Equipment            F                0.042622
                     M                0.070081
Moving violation     F                0.036205
                     M                0.059831
Other                F                0.056522
                     M                0.047146
Registration/plates  F                0.066140
                     M                0.110376
Seat belt            F                0.012598
                     M                0.037980
Speeding             F                0.008720
                     M                0.024925
Name: search_conducted, dtype: float64

## 4. Why is search_type missing so often?

In [42]:
ri.search_type.isnull().value_counts()

True     88545
False     3196
Name: search_type, dtype: int64

In [43]:
ri.search_conducted.value_counts()

False    88545
True      3196
Name: search_conducted, dtype: int64

In [44]:
ri[ri.search_conducted == False].search_type.value_counts()

Series([], Name: search_type, dtype: int64)

In [45]:
ri[ri.search_conducted == False].search_type.value_counts(dropna=False)


NaN    88545
Name: search_type, dtype: int64

When is not search_conducted, is null on search_type

## 5. During a search, how often is the driver frisked?

Lessons: 

- Use string methods to find partial matches
- Use the correct denominator when calculating rates
- Pandas calculations ignore missing values
- Apply the "smell test" to your results

In [47]:
ri['frisk'] = ri.search_type.str.contains('Protective Frisk')

In [48]:
ri['frisk'].value_counts(dropna=False)

NaN      88545
False     2922
True       274
Name: frisk, dtype: int64

In [49]:
ri.frisk.mean()

0.08573216520650813

## 6. Which year had the least number of stops?

Lessons:

- Consider removing chunks of data that may be biased
- Use the datetime data type for dates and times

In [63]:
ri.columns

Index(['stop_date', 'stop_time', 'driver_gender', 'driver_age_raw',
       'driver_age', 'driver_race', 'violation_raw', 'violation',
       'search_conducted', 'search_type', 'stop_outcome', 'is_arrested',
       'stop_duration', 'drugs_related_stop', 'frisk'],
      dtype='object')

In [70]:
ri['stop_date'].str.slice(0, 4).value_counts(normalize=True)

2012    0.119576
2006    0.115968
2007    0.103291
2014    0.100588
2008    0.095399
2015    0.093731
2011    0.088575
2013    0.086374
2009    0.086199
2010    0.082417
2005    0.027883
Name: stop_date, dtype: float64

In [71]:
pd.to_datetime(ri.stop_date)

0       2005-01-02
1       2005-01-18
2       2005-01-23
3       2005-02-20
4       2005-03-14
           ...    
91736   2015-12-31
91737   2015-12-31
91738   2015-12-31
91739   2015-12-31
91740   2015-12-31
Name: stop_date, Length: 91741, dtype: datetime64[ns]

In [75]:
combined = ri.stop_date.str.cat(ri.stop_time, sep=' ')

In [78]:
ri['stop_datetime'] = pd.to_datetime(combined)

In [79]:
ri.stop_datetime.dt.year.value_counts()

2012    10970
2006    10639
2007     9476
2014     9228
2008     8752
2015     8599
2011     8126
2013     7924
2009     7908
2010     7561
2005     2558
Name: stop_datetime, dtype: int64